In [1]:
import pandas as pd
import yfinance as yf
import sqlalchemy
import yfinance as yf
import datetime
import ccxt
import os
import time
import pandas_ta as ta
from backtesting import Backtest, Strategy
import numpy as np
from backtesting.lib import resample_apply
from pandas_datareader import data

C:\Users\ahsua\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
start_date = '2019-01-03'
end_date = '2022-12-31'
df = data.DataReader("TSKB.IS", "yahoo", start_date, end_date)
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-01-03,0.77,0.72,0.76,0.73,59731247.0,0.716256
2019-01-04,0.74,0.71,0.73,0.72,86567376.0,0.706444
2019-01-07,0.74,0.71,0.72,0.74,34822723.0,0.726067
2019-01-08,0.74,0.72,0.73,0.73,19379007.0,0.716256
2019-01-09,0.74,0.72,0.73,0.72,15520401.0,0.706444
...,...,...,...,...,...,...
2022-11-21,3.47,3.29,3.44,3.44,262586955.0,3.440000
2022-11-22,3.54,3.40,3.46,3.50,261480957.0,3.500000
2022-11-23,3.85,3.46,3.50,3.85,405791702.0,3.850000


In [5]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

def sup1(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=1.0)
def sup2(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=2.0)
def sup3(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=3.0)


class RsiOscillator(TrailingStrategy):

    # Do as much initial computation as possible
    def init(self):
        
        super().init()
        super().set_trailing_sl(1)
        
        self.atr= self.I(ta.atr, high= self.data.High.s,low= self.data.Low.s, 
                            close= self.data.Close.s, length=10)
        self.super1 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=1.0, plot=True, overlay=True)
        self.super2 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=2.0,  plot=True, overlay=True)
        self.super3 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=3.0,  plot=True, overlay=True)
        self.super1_w = resample_apply(
            'W-FRI',sup1, self.data.df,plot=False)
        self.super2_w = resample_apply(
            'W-FRI',sup2, self.data.df, plot=False)
        self.super3_w = resample_apply(
            'W-FRI',sup3, self.data.df,plot=False)        
        self.ema20 = self.I(ta.ema, close= self.data.Close.s, length=20, plot=False)
        self.ema20_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=20, plot=False)
        self.ema50 = self.I(ta.ema, close= self.data.Close.s, length=50)
        self.ema200 = self.I(ta.ema, close= self.data.Close.s, length=200)
        self.ema50_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=50, plot=False)
        self.macd = self.I(ta.macd, close= self.data.Close.s, plot=False)
        self.macd_w = resample_apply('W-FRI',ta.macd, self.data.Close.s, plot=False)
    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        
        super().next()
        price = self.data.Close.s[-1]
        atr = self.atr[-1]
        super1=self.super1[0]
        super2=self.super2[0]
        super3=self.super3[0]
        super1w=self.super1_w[0]
        super2w=self.super2_w[0]
        super3w=self.super3_w[0]
        macdh=self.macd[1]
        macdh_w=self.macd_w[1]
        if (price>self.ema50_w[-1]):
            if (super2[-2]==super2[-1]) and (super3[-2]==super3[-1])\
            and price>super1[-1]:
                #self.position.close()
                self.buy(size=1, sl=price-atr)
        #if price<self.super1[-1]:
        #        self.position.close()
        #        #self.sell()

bt = Backtest(df, RsiOscillator, cash=5000, commission=.002)

stats = bt.run()

bt.plot(plot_volume=False)
stats
#print(stats[['# Trades','Win Rate [%]','Best Trade [%]','Worst Trade [%]','Profit Factor','Max. Drawdown [%]','Max. Drawdown Duration']])
#(super2[-2]==super2[-1]) and (super3[-2]==super3[-1]) and macdh_w[-1]>macdh_w[-2] and
#(super2w[-2]==super2w[-1]) and (super3w[-2]==super3w[-1]):
#macdh[-1]>macdh[-2]

Start                     2019-01-03 00:00:00
End                       2022-11-25 00:00:00
Duration                   1422 days 00:00:00
Exposure Time [%]                   28.296146
Equity Final [$]                  5014.402574
Equity Peak [$]                   5014.602574
Return [%]                           0.288051
Buy & Hold Return [%]              434.246574
Return (Ann.) [%]                    0.073541
Volatility (Ann.) [%]                0.066625
Sharpe Ratio                         1.103811
Sortino Ratio                        2.417577
Calmar Ratio                         1.737048
Max. Drawdown [%]                   -0.042337
Avg. Drawdown [%]                    -0.01442
Max. Drawdown Duration      559 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                  138
Win Rate [%]                        41.304348
Best Trade [%]                      79.829028
Worst Trade [%]                      -7.83609
Avg. Trade [%]                    

In [11]:
help(ta.macd)

Help on function macd in module pandas_ta.momentum.macd:

macd(close, fast=None, slow=None, signal=None, talib=None, offset=None, **kwargs)
    Moving Average Convergence Divergence (MACD)
    
    The MACD is a popular indicator to that is used to identify a security's trend.
    While APO and MACD are the same calculation, MACD also returns two more series
    called Signal and Histogram. The Signal is an EMA of MACD and the Histogram is
    the difference of MACD and Signal.
    
    Sources:
        https://www.tradingview.com/wiki/MACD_(Moving_Average_Convergence/Divergence)
        AS Mode: https://tr.tradingview.com/script/YFlKXHnP/
    
    Calculation:
        Default Inputs:
            fast=12, slow=26, signal=9
        EMA = Exponential Moving Average
        MACD = EMA(close, fast) - EMA(close, slow)
        Signal = EMA(MACD, signal)
        Histogram = MACD - Signal
    
        if asmode:
            MACD = MACD - Signal
            Signal = EMA(MACD, signal)
          

In [12]:
deneme=ta.macd(df.Close)
deneme

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
2019-01-03,NaN,NaN,NaN
2019-01-04,NaN,NaN,NaN
2019-01-07,NaN,NaN,NaN
2019-01-08,NaN,NaN,NaN
2019-01-09,NaN,NaN,NaN
...,...,...,...
2022-10-12,-18.404629,-6.237499,-12.167131
2022-10-13,-18.649420,-5.185832,-13.463589
2022-10-14,-19.963266,-5.199742,-14.763524
2022-10-17,-19.619603,-3.884863,-15.734740


In [13]:
deneme2= ta.supertrend(df.High, df.Low, df.Close, length=10, multiplier=1.0)
deneme2

,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0
2019-01-03,0.000000,1,NaN,NaN
2019-01-04,NaN,1,NaN,NaN
2019-01-07,NaN,1,NaN,NaN
2019-01-08,NaN,1,NaN,NaN
2019-01-09,NaN,1,NaN,NaN
...,...,...,...,...
2022-10-12,228.163041,-1,NaN,228.163041
2022-10-13,227.764242,-1,NaN,227.764242
2022-10-14,227.764242,-1,NaN,227.764242
2022-10-17,227.764242,-1,NaN,227.764242


In [14]:
help(ta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, ddof=0, mamode=None, talib=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator by John Bollinger.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=5, std=2, mamode="sma", ddof=0
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length, ddof)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
        PERCENT = (close - LOWER) / (UPPER - LOWER)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 5
        std (int): The long 

In [16]:
bbands = ta.bbands(close = df.Close, std = 1)
bbands

,BBL_5_1.0,BBM_5_1.0,BBU_5_1.0,BBB_5_1.0,BBP_5_1.0
2019-01-03,NaN,NaN,NaN,NaN,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN
2019-01-07,NaN,NaN,NaN,NaN,NaN
2019-01-08,NaN,NaN,NaN,NaN,NaN
2019-01-09,20.726126,21.691867,22.657607,8.904171,0.953952
...,...,...,...,...,...
2022-10-12,215.799657,223.580005,231.360353,6.959789,0.092563
2022-10-13,217.449548,220.298004,223.146461,2.586003,0.749608
2022-10-14,210.328390,216.682004,223.035618,5.864459,-0.420106
2022-10-17,210.181759,215.960004,221.738248,5.351217,0.793342


In [19]:
bdata= []
index2 = 1
bsymbols1=pd.read_csv('bsymbols.csv',header=None)
bsymbols=bsymbols1.iloc[:,0].to_list()
for bticker in bsymbols[:5]:
    print(index2,bticker,end="\r")
    index2 += 1
    start_date = '2019-01-03'
    end_date = '2022-12-31'
    df = data.DataReader(bticker, "yahoo", start_date, end_date)
    bdata.append(df)

In [69]:
for bticker,df in zip (bsymbols[:5], bdata[:5]):
        print(bticker)
        def sup1(fr):
            return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=1.0)
        def sup2(fr):
            return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=2.0)
        def sup3(fr):
            return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=3.0)
        class RsiOscillator(TrailingStrategy):
        
            # Do as much initial computation as possible
            def init(self):
                
                super().init()
                super().set_trailing_sl(1)
                
                self.atr= self.I(ta.atr, high= self.data.High.s,low= self.data.Low.s, 
                                     close= self.data.Close.s, length=10)
                self.super1 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                                     close= self.data.Close.s, length=10, multiplier=1.0, plot=True, overlay=True)
                self.super2 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                                     close= self.data.Close.s, length=10, multiplier=2.0,  plot=True, overlay=True)
                self.super3 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                                     close= self.data.Close.s, length=10, multiplier=3.0,  plot=False, overlay=True)
                self.super1_w = resample_apply(
                    'W-FRI',sup1, self.data.df,plot=False)
                self.super2_w = resample_apply(
                    'W-FRI',sup2, self.data.df, plot=False)
                self.super3_w = resample_apply(
                    'W-FRI',sup3, self.data.df,plot=False)        
                self.ema20 = self.I(ta.ema, close= self.data.Close.s, length=20, plot=False)
                self.ema20_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=20, plot=False)
                self.ema50 = self.I(ta.ema, close= self.data.Close.s, length=50)
                self.ema200 = self.I(ta.ema, close= self.data.Close.s, length=200)
                self.ema50_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=50, plot=False)
                self.macd = self.I(ta.macd, close= self.data.Close.s, plot=False)
                self.macd_w = resample_apply('W-FRI',ta.macd, self.data.Close.s, plot=False)
            # Step through bars one by one
            # Note that multiple buys are a thing here
            def next(self):
                
                super().next()
                price = self.data.Close.s[-1]
                atr = self.atr[-1]
                super1=self.super1[0]
                super2=self.super2[0]
                super3=self.super3[0]
                super1w=self.super1_w[0]
                super2w=self.super2_w[0]
                super3w=self.super3_w[0]
                macdh=self.macd[1]
                macdh_w=self.macd_w[1]
                if (price>self.ema50_w[-1]):
                    if (super2[-2]==super2[-1]) and (super3[-2]==super3[-1])\
                    and price>super1[-1]:
                        #self.position.close()
                        self.buy(size=1, sl=price-atr)
                #if price<self.super1[-1]:
                #        self.position.close()
                #        #self.sell()
        
        bt = Backtest(df, RsiOscillator, cash=700, commission=.002)
        stats = bt.run()
        stats['_trades']
        bt.plot(plot_volume=False)
        #print(stats[['Trades','Win Rate [%]','Best Trade [%]','Worst Trade [%]','Profit Factor','Max. Drawdown [%]','Max. Drawdown Duration']])
        #(super2[-2]==super2[-1]) and (super3[-2]==super3[-1]) and macdh_w[-1]>macdh_w[-2] and
        #(super2w[-2]==super2w[-1]) and (super3w[-2]==super3w[-1]):
        #macdh[-1]>macdh[-2]

VESTL.IS


VAKBN.IS


ULKER.IS


TURSG.IS


TUPRS.IS


Strateji 1
                if (price>self.ema50[-1] or price>self.ema200[-1]) and price>super1w[-1]:
                    if price>super2[-1] and self.data.Close.s[-2]<super2[-2]:
strateji 2

                if (self.ema20[-1]>self.ema50[-1] and price>self.ema200[-1]) and price>super1w[-1]:
                    if (super1[-2]==super1[-1]) and (super2[-2]==super2[-1]) and (super3[-2]==super3[-1])\
                    and price>super1[-1]>super2[-1]>super3[-1]:
strateji3
            price>super2[-1] and self.data.Close.s[-2]<super2[-2]    
            
strateji 4
             if (price>self.ema50_w[-1]):
                    if (super2[-2]==super2[-1]) and (super3[-2]==super3[-1])\
                    and price>super1[-1]:

In [57]:
help(ta.atr)

Help on function atr in module pandas_ta.volatility.atr:

atr(high, low, close, length=None, mamode=None, talib=None, drift=None, offset=None, **kwargs)
    Average True Range (ATR)
    
    Averge True Range is used to measure volatility, especially volatility caused by
    gaps or limit moves.
    
    Sources:
        https://www.tradingview.com/wiki/Average_True_Range_(ATR)
    
    Calculation:
        Default Inputs:
            length=14, drift=1, percent=False
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        WMA = Weighted Moving Average
        RMA = WildeR's Moving Average
        TR = True Range
    
        tr = TR(high, low, close, drift)
        if 'ema':
            ATR = EMA(tr, length)
        elif 'sma':
            ATR = SMA(tr, length)
        elif 'wma':
            ATR = WMA(tr, length)
        else:
            ATR = RMA(tr, length)
    
        if percent:
            ATR *= 100 / close
    
    Args:
        high (pd.Series):

In [73]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    # Do as much initial computation as possible
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)

    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000, commission=.002)
stats = bt.run()

bt.plot()
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,27,373,422,362.67390,439.40,2071.60470,0.211557,2006-02-10,2006-04-24,73 days
1,33,493,525,365.70996,415.46,1641.75132,0.136037,2006-08-03,2006-09-19,47 days
2,24,862,925,561.83142,557.94,-93.39408,-0.006926,2008-01-23,2008-04-23,91 days
3,29,987,1126,467.65344,371.28,-2794.82976,-0.206079,2008-07-22,2009-02-09,202 days
4,19,1367,1424,547.68318,563.00,291.01958,0.027967,2010-01-25,2010-04-16,81 days
5,22,1437,1534,501.98196,512.86,239.31688,0.021670,2010-05-05,2010-09-22,140 days
6,19,1653,1740,573.94560,592.49,352.34360,0.032310,2011-03-14,2011-07-18,126 days
7,20,1873,1913,573.12396,646.60,1469.52080,0.128203,2012-01-26,2012-03-23,57 days
8,20,2072,2129,655.95930,767.69,2234.61400,0.170332,2012-11-09,2013-02-04,87 days
